In [2]:
val reportFolder = "reports"
val executionRounds = 10

In [3]:
import java.io.File
import java.util.Properties

val properties = Properties()
File("gradle.properties").bufferedReader().use {
    properties.load(it)
}

val executedTask = properties.getProperty("scenario.task") ?: "assemble"

val projectPath = properties.getProperty("project.path") ?: properties.getProperty("project.git.directory")

val projectName: String = projectPath?.substringAfterLast("\\")?.substringAfterLast("/")?.removeSuffix(".git") ?: "User"

In [4]:
enum class CompilerMetrics(val readableName: String) {
    GRADLE_TASK("Total Gradle task time"),

    COMPILATION_ROUND("Sources compilation round"),
    
    //compiler metrics
    COMPILER_INITIALIZATION("Compiler initialization time"),
    CODE_ANALYSIS("Compiler code analysis"),
    IR_TRANSLATION("Compiler IR translation"),
    IR_LOWERING("Compiler IR lowering"),
    IR_GENERATION("Compiler IR generation"),
    ;
}

In [5]:
%use dataframe

In [6]:
import java.io.File
import org.jetbrains.kotlinx.dataframe.DataFrame

//Json report print time in milliseconds 
fun readJsonReportFile(file: File): Map<CompilerMetrics, Long?>? {
    if (file.extension != "json" || file.readText().isBlank())
        return null
    val report = DataFrame.read(file)

    if ((report.get("startParameters").get("tasks")[0] as List<String>).get(0) != executedTask)
        return null

    val timeMetrics = report.get("aggregatedMetrics").get("buildTimes").get("buildTimesNs")[0] as DataRow<*>

    return CompilerMetrics.values().associate {
        val value = timeMetrics.getOrNull(it.name)?.let {
            when (it) {
                is Int -> it.toLong()
                is Long -> it
                is String -> it.toLong()
                else -> throw IllegalStateException("Unknown type ${it.javaClass} to convert to Long")
            }
        }
        Pair(it, value)
    }
}

In [8]:
import kotlin.streams.toList

//Text report print time in seconds with 2 decemal digits, so there is an error during conversion to nanoSeconds
fun readTextReportFile(file: File): Map<CompilerMetrics, Long?>? {
    val isValidReportFile = file.extension == "txt" && file.readText().isNotEmpty()  && file.readText().contains("tasks = [$executedTask]")
    if (!isValidReportFile) return null
    
    val aggregatedTimeMetrics = file.bufferedReader().use {
        it.lines().dropWhile { !it.startsWith("Time metrics:") }.takeWhile { it.isNotBlank() }.toList()
    }

    return CompilerMetrics.values().associate { metric ->
        val metricLine = aggregatedTimeMetrics.firstOrNull { it.trim().startsWith(metric.readableName) }
        val value = metricLine?.substringAfter(": ")?.substringBefore(" ")?.replace(",", "")?.toDouble()
        Pair(metric, value?.times(1_000_000_000)?.toLong())
    }
}


# Read compiler metrics


In [9]:
%use kandy

In [10]:
import org.jetbrains.kotlinx.dataframe.api.toDataFrame
import java.nio.file.Files
import kotlin.io.path.Path

fun readMetrics(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>?
) = directory.listFiles()
    .sortedByDescending(File::lastModified)
    .mapNotNull { readReportFile(it) }
    .take(executionRounds)
    .toList()

fun collectMeanValue(
    metrics: List<Map<CompilerMetrics, Long?>>,
): Map<String, Any?> {
    val rows = CompilerMetrics.values().map { compilerMetric ->
        val values = metrics.map { it[compilerMetric] }.filterNotNull()
        Pair(compilerMetric.name, values)
    }.filter { 
        val allValuesPresent = (it.second.size == executionRounds)
        if (!allValuesPresent) {
            println("Unable to calculate mean value for ${it.first}, only ${it.second.size} runs are found")
        }
        allValuesPresent
    }.toTypedArray()

    val meanValue = dataFrameOf(*rows).mean()

    return CompilerMetrics.values().associate {
        Pair(it.name, meanValue.getOrNull(it.name))
    }
} 


# Collect data into dataframe

In [12]:
val kotlinVersionColumn = "kotlinVersion"
val buildScenarioColumn = "buildScenario"

val columnNames = CompilerMetrics.values().map { it.name }.toMutableList()
columnNames.add(kotlinVersionColumn)
columnNames.add(buildScenarioColumn)

fun collectResultsIntoDataFrame(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>?,
): AnyFrame {
    val rows = directory.listFiles().filter { it.isDirectory() }.flatMap { versionDir ->
        versionDir.listFiles().filter { it.isDirectory() }.map { scenarioDir ->
            val buildMetrics = readMetrics(scenarioDir, readReportFile)
            //rename scenarios for graphics only
            val scenario = when (scenarioDir.name) {
                "clean_build" -> "Clean build"
                "incremental_abi_build" -> "Incremental build for ABI changes"
                "incremental_non_abi_build" -> "Incremental build for non-ABI changes"
                else -> scenarioDir.name
            }

            println("\n Validate \'$scenario\' scenario for ${versionDir.name} kotlin version")
            val row = collectMeanValue(buildMetrics).toMutableMap()
            row.put(kotlinVersionColumn, versionDir.name)
            row.put(buildScenarioColumn, scenario)
            row
        }
    }

    val db = columnNames.map { compilerMetric -> Pair(compilerMetric, rows.map { it[compilerMetric] })}.toTypedArray()
    return dataFrameOf(*db)
}

In [13]:
import java.util.concurrent.TimeUnit
import kotlin.time.DurationUnit
import kotlin.time.toDuration

enum class TimeTransformation(val printTime: String, val transformation: (Double?) -> Double?) {
    NANOSECONDS("nanosaconds", { it }),
    MILLISECONDS("milliseconds", { 
        it?.toDuration(DurationUnit.NANOSECONDS)?.toDouble(DurationUnit.MILLISECONDS) 
    }),
    SECONDS("seconds", {
        it?.toDuration(DurationUnit.NANOSECONDS)?.toDouble(DurationUnit.SECONDS) 
    })
    ;
}

fun getTimeTranformationForColumn(df: DataFrame<Any?>, column: String): TimeTransformation {
    val meanValue = df.mean(column, skipNA = true)
    val ratio = 1_000
    return when {
        meanValue < 10 * ratio -> TimeTransformation.NANOSECONDS
        meanValue < 10 * ratio * ratio -> TimeTransformation.MILLISECONDS
        else -> TimeTransformation.SECONDS
    }
}

In [14]:
var df = collectResultsIntoDataFrame(File(reportFolder), ::readTextReportFile)
    .sortBy(kotlinVersionColumn)
val timeTransformation = getTimeTranformationForColumn(df, CompilerMetrics.COMPILATION_ROUND.name)

 CompilerMetrics.values().forEach { 
     df = df.update(it.name).with { timeTransformation.transformation(it as Double) }
 }

val dataFrame = df
val scenarios = dataFrame[buildScenarioColumn]


 Validate 'Incremental build for ABI changes' scenario for 1.9.23 kotlin version

 Validate 'Incremental build for non-ABI changes' scenario for 1.9.23 kotlin version

 Validate 'Clean build' scenario for 1.9.23 kotlin version

 Validate 'Incremental build for ABI changes' scenario for 2.0.0-RC1 kotlin version

 Validate 'Incremental build for non-ABI changes' scenario for 2.0.0-RC1 kotlin version

 Validate 'Clean build' scenario for 2.0.0-RC1 kotlin version


In [34]:
val plots = scenarios.distinct().toList().map { scenario ->
    dataFrame.filter { buildScenario == scenario }.plot {
        layout {
            title = "$projectName project"
            subtitle = "$scenario"
            style { 
                legend {
                    position = LegendPosition.Top
                }
            }
        }
        bars {
            x(kotlinVersionColumn) {
                axis.name = "Gradle compilation time"
            }
            y(CompilerMetrics.COMPILATION_ROUND.name) {
                axis.name = "Time (${timeTransformation.printTime})"
            }
        }
        
    }
}
plotGrid(plots, 1)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="xgH6nI"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"layout":{
"name":"grid",
"ncol":1,
"nrow":3,
"fit":true,
"align":false
},
"figures":[{
"ggtitle":{
"text":"kmath-2 project",
"subtitle":"Incremental build for ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"COMPILATION_ROUND":[0.958,0.942]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle compilation time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"kotlinVersion",
"y":"COMPILATION_ROUND"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
},{
"ggtitle":{
"text":"kmath-2 project",
"subtitle":"Incremental build for non-ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"COMPILATION_ROUND":[0.99,0.966]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle compilation time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"kotlinVersion",
"y":"COMPILATION_ROUND"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
},{
"ggtitle":{
"text":"kmath-2 project",
"subtitle":"Clean build"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"COMPILATION_ROUND":[14.662,12.807]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle compilation time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"kotlinVersion",
"y":"COMPILATION_ROUND"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
}],
"kind":"subplots"
};
 var plotContainer = document.getElementById("xgH6nI");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project 
 
 
 
 
 Incremental build for ABI changes 
 
 
 
 
 Time (seconds) 
 
 
 
 
 Gradle compilation time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.5 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project 
 
 
 
 
 Incremental build for non-ABI changes 
 
 
 
 
 Time (seconds) 
 
 
 
 
 Gradle compilation time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project 
 
 
 
 
 Clean build 
 
 
 
 
 Time (seconds) 
 
 
 
 
 Gradle compilation time

In [16]:
val incrementalScenatiosLabels = listOf(Pair("Incremental build for ABI changes", "ABI changes"), Pair("Incremental build for non-ABI changes", "non-ABI changes"))

In [35]:
fun gradleBuildTime(scenarios: List<String>, name: String, scenarioBreaksLabeled: List<Pair<String, String>>? = null) =
    dataFrame.filter { buildScenario in scenarios }
        .plot {
            layout {
                title = "$projectName project - $name"
                style {
                    legend {
                        position = LegendPosition.Top
                    }
                }
            }
            bars {
                x(buildScenarioColumn) {
                    axis.name = "Gradle build time"
                    scenarioBreaksLabeled?.let {
                        axis.breaksLabeled(it.map { it.first }, it.map { it.second })
                    }
                }
                y(GRADLE_TASK.name) {
                    axis.name = "Time (${timeTransformation.printTime})"
                }
                fillColor(kotlinVersionColumn) {
                    legend.name = ""
                }
            }
        }

In [36]:
gradleBuildTime(listOf("Clean build"), "Clean build scenario")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="DFswjP"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project - Clean build scenario"
},
"mapping":{
},
"data":{
"GRADLE_TASK":[76.267,72.873],
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"buildScenario":["Clean build","Clean build"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Gradle build time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"GRADLE_TASK",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("DFswjP");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Clean build 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project - Clean build scenario 
 
 
 
 
 Time (seconds) 
 
 
 
 
 Gradle build time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

In [37]:
gradleBuildTime(listOf("Incremental build for ABI changes", "Incremental build for non-ABI changes"), "Incremental build scenarios", incrementalScenatiosLabels)


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="DWfAHW"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project - Incremental build scenarios"
},
"mapping":{
},
"data":{
"GRADLE_TASK":[5.503,5.454,5.557,5.368],
"kotlinVersion":["1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1"],
"buildScenario":["Incremental build for ABI changes","Incremental build for non-ABI changes","Incremental build for ABI changes","Incremental build for non-ABI changes"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"breaks":["Incremental build for ABI changes","Incremental build for non-ABI changes"],
"name":"Gradle build time",
"labels":["ABI changes","non-ABI changes"]
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"GRADLE_TASK",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("DWfAHW");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ABI changes 
 
 
 
 
 
 
 
 
 non-ABI changes 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project - Incremental build scenarios 
 
 
 
 
 Time (seconds) 
 
 
 
 
 Gradle build time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

In [39]:
fun kotlinCompilationTime(
    scenarios: List<String>,
    name: String,
    scenarioBreaksLabeled: List<Pair<String, String>>? = null
) = dataFrame.filter { buildScenario in scenarios }
    .plot {
        layout {
            title = "$projectName project - $name"
            style {
                legend {
                    position = LegendPosition.Top
                }
            }
        }
        bars {
            x(buildScenarioColumn) {
                axis.name = "Kotlin compilation time"
                scenarioBreaksLabeled?.let {
                    axis.breaksLabeled(it.map { it.first }, it.map { it.second })
                }
            }
            y(COMPILATION_ROUND.name) {
                axis.name = "Time (${timeTransformation.printTime})"
            }
            fillColor(kotlinVersionColumn) {
                legend.name = ""
            }
        }
    }
    


In [40]:
kotlinCompilationTime(listOf("Clean build"), "Clean build scenario")


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="hBVBhf"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project - Clean build scenario"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","2.0.0-RC1"],
"buildScenario":["Clean build","Clean build"],
"COMPILATION_ROUND":[14.662,12.807]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Kotlin compilation time"
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"COMPILATION_ROUND",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("hBVBhf");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Clean build 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project - Clean build scenario 
 
 
 
 
 Time (seconds) 
 
 
 
 
 Kotlin compilation time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

In [41]:
kotlinCompilationTime(listOf("Incremental build for ABI changes", "Incremental build for non-ABI changes"), "Incremental build scenarios", incrementalScenatiosLabels)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="4Ym9mt"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project - Incremental build scenarios"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1"],
"buildScenario":["Incremental build for ABI changes","Incremental build for non-ABI changes","Incremental build for ABI changes","Incremental build for non-ABI changes"],
"COMPILATION_ROUND":[0.958,0.99,0.942,0.966]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"breaks":["Incremental build for ABI changes","Incremental build for non-ABI changes"],
"name":"Kotlin compilation time",
"labels":["ABI changes","non-ABI changes"]
},{
"aesthetic":"y",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"name":""
}],
"layers":[{
"mapping":{
"x":"buildScenario",
"y":"COMPILATION_ROUND",
"fill":"kotlinVersion"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}],
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("4Ym9mt");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ABI changes 
 
 
 
 
 
 
 
 
 non-ABI changes 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 
 
 
 
 0.2 
 
 
 
 
 
 
 0.4 
 
 
 
 
 
 
 0.6 
 
 
 
 
 
 
 0.8 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project - Incremental build scenarios 
 
 
 
 
 Time (seconds) 
 
 
 
 
 Kotlin compilation time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.0.0-RC1

# Compare Gradle and Compiler time

In [49]:
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalContinuousScale
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalScale
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalTransform

fun getPlotForCompilerMetrics(
    scenario: String,
    graphicsCaption: String,
    vararg metrics: Pair<CompilerMetrics, String>
) = dataFrame
    .gather(*metrics.map { it.first.name }.toTypedArray()).into("metric", "time_ns")
    .filter { it["buildScenario"] == scenario }
    .plot {
        layout {
            title = "$projectName project"
            subtitle = scenario
            caption = graphicsCaption
            style {
                legend {
                    position = LegendPosition.Top
                }
            }
            size = Pair(800, 300)
        }
        barsH {
            y("kotlinVersion"<String>()) {
                axis.name = "Kotlin version"
            }
            x("time_ns"<Double>()) {
                axis.name = "Time (${timeTransformation.printTime})"
            }
            fillColor("metric"<String>()) {
                legend {
                    name = ""
                    breaksLabeled(*metrics.map { Pair(it.first.name, it.second) }.toTypedArray())
                }
            }
            position = Position.stack()

        }
    }

In [44]:
val versionByScenario =
    scenarios.distinct().toList().associate { name -> Pair(name as String, getPlotForCompilerMetrics(name, "Gradle compilation breakdown", CompilerMetrics.COMPILATION_ROUND to "Kotlin Compiler time", 
        CompilerMetrics.GRADLE_TASK to "Gradle time")) }

In [45]:
versionByScenario.get("Clean build")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="yyPScv"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project",
"subtitle":"Clean build"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILATION_ROUND","GRADLE_TASK","COMPILATION_ROUND","GRADLE_TASK"],
"time_ns":[14.662,76.267,12.807,72.873]
},
"ggsize":{
"width":800.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"breaks":["COMPILATION_ROUND","GRADLE_TASK"],
"name":"",
"labels":["Kotlin Compiler time","Gradle time"]
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Gradle compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("yyPScv");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 70 
 
 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 
 
 90 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project 
 
 
 
 
 Clean build 
 
 
 
 
 Kotlin version 
 
 
 
 
 Time (seconds) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Compiler time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Gradle time 
 
 
 
 
 
 
 
 
 Gradle compilation breakdown

In [46]:
versionByScenario.get("Incremental build for non-ABI changes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="rHrdVy"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project",
"subtitle":"Incremental build for non-ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILATION_ROUND","GRADLE_TASK","COMPILATION_ROUND","GRADLE_TASK"],
"time_ns":[0.99,5.454,0.966,5.368]
},
"ggsize":{
"width":800.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"breaks":["COMPILATION_ROUND","GRADLE_TASK"],
"name":"",
"labels":["Kotlin Compiler time","Gradle time"]
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Gradle compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("rHrdVy");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project 
 
 
 
 
 Incremental build for non-ABI changes 
 
 
 
 
 Kotlin version 
 
 
 
 
 Time (seconds) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Compiler time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Gradle time 
 
 
 
 
 
 
 
 
 Gradle compilation breakdown

In [47]:
versionByScenario.get("Incremental build for ABI changes")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Ttd2cD"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project",
"subtitle":"Incremental build for ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILATION_ROUND","GRADLE_TASK","COMPILATION_ROUND","GRADLE_TASK"],
"time_ns":[0.958,5.503,0.942,5.557]
},
"ggsize":{
"width":800.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"breaks":["COMPILATION_ROUND","GRADLE_TASK"],
"name":"",
"labels":["Kotlin Compiler time","Gradle time"]
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Gradle compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("Ttd2cD");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project 
 
 
 
 
 Incremental build for ABI changes 
 
 
 
 
 Kotlin version 
 
 
 
 
 Time (seconds) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Kotlin Compiler time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Gradle time 
 
 
 
 
 
 
 
 
 Gradle compilation breakdown

# Collect Compiler metrics

In [50]:
val compilerMetrics = scenarios.distinct().toList().associate {
    Pair(
        it as String, getPlotForCompilerMetrics(
            it, "Kotlin compilation breakdown",
            CompilerMetrics.COMPILER_INITIALIZATION to "Initialization phase",
            CompilerMetrics.CODE_ANALYSIS to "Analysis phase",
            CompilerMetrics.IR_TRANSLATION to "Translation phase",
            CompilerMetrics.IR_LOWERING to "Lowering",
            CompilerMetrics.IR_GENERATION to "Generation"
        )
    )
}

In [51]:
compilerMetrics.get("Clean build") 


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="EgHbhJ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project",
"subtitle":"Clean build"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","1.9.23","1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION","COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"time_ns":[0.721,3.78,1.271,1.242,1.774,0.621,3.227,1.02,0.994,1.724]
},
"ggsize":{
"width":800.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"breaks":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"name":"",
"labels":["Initialization phase","Analysis phase","Translation phase","Lowering","Generation"]
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Kotlin compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("EgHbhJ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project 
 
 
 
 
 Clean build 
 
 
 
 
 Kotlin version 
 
 
 
 
 Time (seconds) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Initialization phase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Analysis phase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Translation phase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Lowering 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Generation 
 
 
 
 
 
 
 
 
 Kotlin compilation breakdown

In [52]:
compilerMetrics.get("Incremental build for non-ABI changes") 

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="2uqwJ3"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project",
"subtitle":"Incremental build for non-ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","1.9.23","1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION","COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"time_ns":[0.02,0.156,0.264,0.09,0.086,0.02,0.158,0.213,0.084,0.083]
},
"ggsize":{
"width":800.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"breaks":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"name":"",
"labels":["Initialization phase","Analysis phase","Translation phase","Lowering","Generation"]
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Kotlin compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("2uqwJ3");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 
 
 
 
 
 
 0.05 
 
 
 
 
 
 
 
 
 0.10 
 
 
 
 
 
 
 
 
 0.15 
 
 
 
 
 
 
 
 
 0.20 
 
 
 
 
 
 
 
 
 0.25 
 
 
 
 
 
 
 
 
 0.30 
 
 
 
 
 
 
 
 
 0.35 
 
 
 
 
 
 
 
 
 0.40 
 
 
 
 
 
 
 
 
 0.45 
 
 
 
 
 
 
 
 
 0.50 
 
 
 
 
 
 
 
 
 0.55 
 
 
 
 
 
 
 
 
 0.60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project 
 
 
 
 
 Incremental build for non-ABI changes 
 
 
 
 
 Kotlin version 
 
 
 
 
 Time (seconds) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Initialization phase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Analysis phase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Translation phase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Lowering 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Generation 
 
 
 
 
 
 
 
 
 Kotlin compilation breakdown

In [53]:
compilerMetrics.get("Incremental build for ABI changes") 

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="vFCDfB"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"kmath-2 project",
"subtitle":"Incremental build for ABI changes"
},
"mapping":{
},
"data":{
"kotlinVersion":["1.9.23","1.9.23","1.9.23","1.9.23","1.9.23","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1","2.0.0-RC1"],
"metric":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION","COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"time_ns":[0.02,0.155,0.267,0.073,0.08,0.019,0.153,0.259,0.072,0.079]
},
"ggsize":{
"width":800.0,
"height":300.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"discrete":true,
"name":"Kotlin version"
},{
"aesthetic":"x",
"name":"Time (seconds)",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true,
"breaks":["COMPILER_INITIALIZATION","CODE_ANALYSIS","IR_TRANSLATION","IR_LOWERING","IR_GENERATION"],
"name":"",
"labels":["Initialization phase","Analysis phase","Translation phase","Lowering","Generation"]
}],
"layers":[{
"mapping":{
"y":"kotlinVersion",
"x":"time_ns",
"fill":"metric"
},
"stat":"identity",
"orientation":"y",
"sampling":"none",
"position":"stack",
"geom":"bar",
"data":{
}
}],
"caption":{
"text":"Kotlin compilation breakdown"
},
"theme":{
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false,
"legend_position":"top"
}
};
 var plotContainer = document.getElementById("vFCDfB");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 
 
 
 
 
 
 0.05 
 
 
 
 
 
 
 
 
 0.10 
 
 
 
 
 
 
 
 
 0.15 
 
 
 
 
 
 
 
 
 0.20 
 
 
 
 
 
 
 
 
 0.25 
 
 
 
 
 
 
 
 
 0.30 
 
 
 
 
 
 
 
 
 0.35 
 
 
 
 
 
 
 
 
 0.40 
 
 
 
 
 
 
 
 
 0.45 
 
 
 
 
 
 
 
 
 0.50 
 
 
 
 
 
 
 
 
 0.55 
 
 
 
 
 
 
 
 
 0.60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.9.23 
 
 
 
 
 
 
 2.0.0-RC1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 kmath-2 project 
 
 
 
 
 Incremental build for ABI changes 
 
 
 
 
 Kotlin version 
 
 
 
 
 Time (seconds) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Initialization phase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Analysis phase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Translation phase 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Lowering 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Generation 
 
 
 
 
 
 
 
 
 Kotlin compilation breakdown